# Práctica 2

In [6]:
# Importación de módulos necesarios
using DataFrames
using CSV
using Statistics
using Random
using HypothesisTests
using StatsBase
using Plots
using MLJ
using MLJBase
using CategoricalArrays
using LinearAlgebra
using Printf

# Configuración de semilla para reproducibilidad
const SEED = 104
Random.seed!(SEED)

println("Paquetes cargados correctamente")

Paquetes cargados correctamente


## PARTE 1: PREPARACIÓN DE LOS DATOS

### 1.1 Carga y unificación de datos

In [7]:
# Función recursiva para encontrar todos los CSV
function all_csv_files(dir::String)
    archivos = String[]
    for entry in readdir(dir; join=true)
        if isdir(entry) # si es un directorio, llamar recursivamente
            append!(archivos, all_csv_files(entry)) 
        elseif endswith(entry, ".csv") # si es un archivo CSV
            push!(archivos, entry)
        end
    end
    return archivos
end

function import_dataset()
    base_path = joinpath(pwd(), "Datos_Práctica_Evaluación")

    # Obtener todos los archivos CSV recursivamente
    rutas = all_csv_files(base_path)
    println("Archivos CSV encontrados: ", length(rutas))
    
    if isempty(rutas)
        error("No se encontraron archivos CSV en la estructura de carpetas.")
    end
    
    # Leer y combinar todos los CSV
    dfs = DataFrame[]
    for (i, ruta) in enumerate(rutas)
        println("  [$i/$(length(rutas))] Leyendo: ", basename(ruta))
        df_temp = CSV.read(ruta, DataFrame)
        push!(dfs, df_temp)
    end

    # Concatenar todos los DataFrames
    df_unificado = vcat(dfs...)
    println("Dimensiones del DataFrame unificado: ", size(df_unificado))
    return df_unificado
end

println("\n" * "-"^80)
println("CARGA Y UNIFICACIÓN DE DATOS")
println("-"^80)

# Importar el dataset unificado
df = import_dataset()



--------------------------------------------------------------------------------
CARGA Y UNIFICACIÓN DE DATOS
--------------------------------------------------------------------------------
Archivos CSV encontrados: 30
  [1/30] Leyendo: Sujeto_01.csv
  [2/30] Leyendo: Sujeto_05.csv
  [3/30] Leyendo: Sujeto_07.csv
  [4/30] Leyendo: Sujeto_11.csv
  [5/30] Leyendo: Sujeto_03.csv
  [6/30] Leyendo: Sujeto_09.csv
  [7/30] Leyendo: Sujeto_23.csv
  [8/30] Leyendo: Sujeto_25.csv
  [9/30] Leyendo: Sujeto_15.csv
  [10/30] Leyendo: Sujeto_17.csv
  [11/30] Leyendo: Sujeto_21.csv
  [12/30] Leyendo: Sujeto_13.csv
  [13/30] Leyendo: Sujeto_19.csv
  [14/30] Leyendo: Sujeto_27.csv
  [15/30] Leyendo: Sujeto_29.csv
  [16/30] Leyendo: Sujeto_02.csv
  [17/30] Leyendo: Sujeto_04.csv
  [18/30] Leyendo: Sujeto_06.csv
  [19/30] Leyendo: Sujeto_08.csv
  [20/30] Leyendo: Sujeto_10.csv
  [21/30] Leyendo: Sujeto_12.csv
  [22/30] Leyendo: Sujeto_14.csv
  [23/30] Leyendo: Sujeto_16.csv
  [24/30] Leyendo: Sujeto_18.

Row,subject,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAcc-std()-X,tGravityAcc-std()-Y,tGravityAcc-std()-Z,tGravityAcc-mad()-X,tGravityAcc-mad()-Y,tGravityAcc-mad()-Z,tGravityAcc-max()-X,tGravityAcc-max()-Y,tGravityAcc-max()-Z,tGravityAcc-min()-X,tGravityAcc-min()-Y,tGravityAcc-min()-Z,tGravityAcc-sma(),tGravityAcc-energy()-X,tGravityAcc-energy()-Y,tGravityAcc-energy()-Z,tGravityAcc-iqr()-X,tGravityAcc-iqr()-Y,tGravityAcc-iqr()-Z,tGravityAcc-entropy()-X,tGravityAcc-entropy()-Y,tGravityAcc-entropy()-Z,"tGravityAcc-arCoeff()-X,1","tGravityAcc-arCoeff()-X,2","tGravityAcc-arCoeff()-X,3","tGravityAcc-arCoeff()-X,4","tGravityAcc-arCoeff()-Y,1","tGravityAcc-arCoeff()-Y,2","tGravityAcc-arCoeff()-Y,3","tGravityAcc-arCoeff()-Y,4","tGravityAcc-arCoeff()-Z,1","tGravityAcc-arCoeff()-Z,2","tGravityAcc-arCoeff()-Z,3","tGravityAcc-arCoeff()-Z,4","tGravityAcc-correlation()-X,Y","tGravityAcc-correlation()-X,Z","tGravityAcc-correlation()-Y,Z",tBodyAccJerk-mean()-X,tBodyAccJerk-mean()-Y,tBodyAccJerk-mean()-Z,tBodyAccJerk-std()-X,tBodyAccJerk-std()-Y,tBodyAccJerk-std()-Z,tBodyAccJerk-mad()-X,tBodyAccJerk-mad()-Y,tBodyAccJerk-mad()-Z,tBodyAccJerk-max()-X,tBodyAccJerk-max()-Y,tBodyAccJerk-max()-Z,tBodyAccJerk-min()-X,tBodyAccJerk-min()-Y,tBodyAccJerk-min()-Z,tBodyAccJerk-sma(),tBodyAccJerk-energy()-X,tBodyAccJerk-energy()-Y,tBodyAccJerk-energy()-Z,⋯
,Int64,Float64,Float64,Float64,Float64?,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64?,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64?,Float64,⋯
1,1,0.288585,-0.0202942,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.94322,-0.407747,-0.679338,missing,0.929294,-0.853011,0.35991,-0.0585264,0.256892,-0.224848,0.264106,-0.0952456,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.66079,0.963396,-0.14084,0.115375,-0.98525,-0.981708,-0.877625,-0.985001,-0.984416,-0.894677,0.892055,-0.161265,0.12466,0.977436,-0.123213,0.0564827,-0.375426,0.899469,-0.970905,-0.97551,-0.984325,-0.988849,-0.917743,-1.0,-1.0,0.113806,-0.590425,0.591146,missing,0.592469,-0.745449,0.720862,-0.712372,0.7113,-0.995112,0.995675,-0.995668,0.991653,0.570222,0.439027,0.986913,0.0779963,0.0050008,-0.0678308,-0.993519,-0.98836,-0.993575,-0.994488,-0.986207,-0.992818,-0.98518,-0.991994,-0.993119,0.989835,0.991957,0.990519,-0.993522,-0.999935,-0.99982,-0.999878,⋯
2,1,0.274262,-0.012

In [8]:
println("\n" * "-"^80)
println("DESCRIPCIÓN DEL DATASET")
println("-"^80)

# Número de variables (columnas)
num_variables = ncol(df)
println("• Número de variables: ", num_variables)

# Número de instancias (filas)
num_instancias = nrow(df)
println("• Número de instancias: ", num_instancias)

# Número de individuos únicos (columna 'subject')
num_individuos = length(unique(df.subject))
println("• Número de individuos: ", num_individuos)

# Columna de salida: la última columna
num_clases_salida = length(unique(df.Activity))
println("• Número de clases de salida: ", num_clases_salida)

clases = unique(df.Activity)
println("• Clases de salida (activity): ", clases)



--------------------------------------------------------------------------------
DESCRIPCIÓN DEL DATASET
--------------------------------------------------------------------------------
• Número de variables: 563
• Número de instancias: 10299
• Número de individuos: 30
• Número de clases de salida: 6
• Clases de salida (activity): String31["STANDING", "SITTING", "LAYING", "WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS"]


### 1.2 Análisis de valores ausentes

In [9]:
function missing_values_summary(df::DataFrame)   
    # Lista de columnas con missing
    columnas_con_missing = String[]
    missing_counts = Dict{String, Int}()
    
    println("\nColumnas con valores ausentes:")
    for col in names(df)
        num_missing = count(ismissing, df[!, col])
        if num_missing > 0
            push!(columnas_con_missing, col)
            missing_counts[col] = num_missing
            pct_missing = num_missing / nrow(df) * 100
            println("  • $(col): $(num_missing) valores ($(round(pct_missing, digits=2))%)")
        end
    end
    
    if isempty(columnas_con_missing)
        println("    No hay columnas con valores ausentes.")
    else
        println("\n  Total de columnas con valores ausentes: ", length(columnas_con_missing))
    end
    
    # Porcentaje de valores ausentes en todo el dataset
    total_missing = count(ismissing, Iterators.flatten(eachcol(df)))
    total_values = nrow(df) * ncol(df)
    pct_total_missing = total_missing / total_values * 100
    println("\n  Porcentaje total de valores ausentes: $(round(pct_total_missing, digits=4))%")
    
    return columnas_con_missing, missing_counts
end

println("\n" * "-"^80)
println("ANÁLISIS DE VALORES AUSENTES")
println("-"^80)
columnas_missing, missing_counts = missing_values_summary(df);


--------------------------------------------------------------------------------
ANÁLISIS DE VALORES AUSENTES
--------------------------------------------------------------------------------

Columnas con valores ausentes:
  • tBodyAcc-std()-X: 2 valores (0.02%)
  • tBodyAcc-std()-Z: 2 valores (0.02%)
  • tBodyAcc-mad()-X: 2 valores (0.02%)
  • tBodyAcc-mad()-Y: 3 valores (0.03%)
  • tBodyAcc-mad()-Z: 3 valores (0.03%)
  • tBodyAcc-max()-X: 2 valores (0.02%)
  • tBodyAcc-max()-Z: 1 valores (0.01%)
  • tBodyAcc-sma(): 1 valores (0.01%)
  • tBodyAcc-iqr()-X: 3 valores (0.03%)
  • tBodyAcc-iqr()-Y: 1 valores (0.01%)
  • tBodyAcc-entropy()-Z: 1029 valores (9.99%)
  • tBodyAcc-arCoeff()-Z,2: 1029 valores (9.99%)
  • tBodyAcc-correlation()-X,Y: 1029 valores (9.99%)
  • tGravityAcc-std()-Y: 1029 valores (9.99%)
  • tGravityAcc-std()-Z: 1029 valores (9.99%)
  • tGravityAcc-mad()-Z: 1029 valores (9.99%)
  • tGravityAcc-max()-X: 1029 valores (9.99%)
  • tGravityAcc-arCoeff()-X,3: 1029 valores (

### 1.3  Tratamiento y transformación de datos

Media por sujeto (subject-wise).
Los datos son MCAR y cada sujeto tiene características únicas.

In [10]:
function subject_wise_mean_imputation(df::DataFrame)
    df_imputed = deepcopy(df)
    subject_col = :subject
    
    # Obtener columnas numéricas con missing
    num_cols = String[]
    for col in names(df)
        if col != string(subject_col) && col != "Activity" && any(ismissing, df[!, col])
            push!(num_cols, col)
        end
    end
    
    println("  Columnas numéricas a imputar: ", length(num_cols))
    
    # Contadores para estadísticas
    total_imputed = 0
    
    for subj in unique(df[!, subject_col])
        idx = findall(==(subj), df[!, subject_col])
        subdf = df[idx, :]
        
        for col in num_cols
            # Contar valores missing en esta columna para este sujeto
            missing_mask = ismissing.(subdf[!, col])
            n_missing = sum(missing_mask)
            
            if n_missing == 0
                continue
            end
            
            # Si todos los valores del sujeto son missing, usar la media global
            if all(missing_mask)
                # Media global (de todos los sujetos que tienen valores)
                global_mean = mean(skipmissing(df[!, col]))
                df_imputed[idx, col] .= global_mean
                total_imputed += n_missing
            else
                # Calcular la media del sujeto
                subject_mean = mean(skipmissing(subdf[!, col]))
                # Reemplazar missing por la media del sujeto
                df_imputed[idx, col] = coalesce.(df_imputed[idx, col], subject_mean)
                total_imputed += n_missing
            end
        end
    end
    
    println("  Valores imputados: ", total_imputed)
    return df_imputed
end

println("\n" * "-"^80)
println("TRATAMIENTO Y TRANSFORMACIÓN DE DATOS")
println("-"^80)

df_imputed = subject_wise_mean_imputation(df)

# Comprobar que no quedan valores ausentes
println("\n  Verificación post-imputación:")
remaining_missing = count(ismissing, Iterators.flatten(eachcol(df_imputed)))
println("  Valores ausentes restantes: ", remaining_missing)
@assert remaining_missing == 0 "Existen valores ausentes"


--------------------------------------------------------------------------------
TRATAMIENTO Y TRANSFORMACIÓN DE DATOS
--------------------------------------------------------------------------------
  Columnas numéricas a imputar: 175
  Valores imputados: 57892

  Verificación post-imputación:
  Valores ausentes restantes: 0


### Separación de datos: X (features), y (target), subject_id

Separa el dataset en:
- X: Matriz de características (Float64)
- y: Vector categórico de etiquetas (CategoricalArray)
- subject_id: Vector de IDs de sujetos


NO se usa one-hot encoding. MLJ trabaja mejor con CategoricalArray

In [11]:
function separate_data(df::DataFrame)
    # 1. Separar el ID del sujeto
    subject_id = df[!, :subject]
    
    # 2. Separar y (Variable Objetivo)
    # Convertir a CategoricalArray (MLJ lo prefiere así)
    y = categorical(df[!, :Activity])
    
    # 3. Separar X (Features)
    # Seleccionar solo columnas numéricas (excluir subject y Activity)
    X_df = DataFrames.select(df, Not([:subject, :Activity]))
    
    # Convertir DataFrame a Matrix para eficiencia
    # MLJ puede trabajar con ambos, pero Matrix es más eficiente para operaciones numéricas
    X_matrix = Matrix{Float64}(X_df)
    
    return X_matrix, y, subject_id
end

println("\n" * "-"^80)
println("SEPARACIÓN DE DATOS")
println("-"^80)

X_full, y_full, subject_id_full = separate_data(df_imputed)
println("\nDatos separados:")
println(" • X (features): $(size(X_full)) - Tipo: $(typeof(X_full))")
println(" • y (target): $(size(y_full)) - Tipo: $(typeof(y_full))")
println(" • subject_id: $(size(subject_id_full))")



--------------------------------------------------------------------------------
SEPARACIÓN DE DATOS
--------------------------------------------------------------------------------

Datos separados:
 • X (features): (10299, 561) - Tipo: Matrix{Float64}
 • y (target): (10299,) - Tipo: CategoricalVector{String31, UInt32, String31, CategoricalValue{String31, UInt32}, Union{}}
 • subject_id: (10299,)


### 1.4 Partición Holdout (subject-wise)

Realiza la partición holdout asegurando que todos los datos de un sujeto están en el mismo conjunto (train o test).

In [12]:
function holdout_split_subject_wise(X::Matrix, y::CategoricalArray, subject_id::AbstractArray; 
                                    test_ratio::Real=0.1)

    # Asegurar reproducibilidad
    Random.seed!(SEED) 
    
    unique_subjects = unique(subject_id)
    num_subjects = length(unique_subjects)
    
    # Seleccionar el 10% de sujetos para test
    n_test = max(1, Int(round(num_subjects * test_ratio)))
    shuffled_subjects = shuffle(unique_subjects)
    test_subjects = shuffled_subjects[1:n_test]
    println("  Individuos seleccionados para TEST (holdout $(round(test_ratio*100))%): ", sort(test_subjects))

    # División de instancias
    is_test = [s in test_subjects for s in subject_id]
    is_train = .!is_test
    
    X_train = X[is_train, :]
    y_train = y[is_train]
    subject_train = subject_id[is_train]
    
    X_test = X[is_test, :]
    y_test = y[is_test]
    subject_test = subject_id[is_test]

    println("\n  Partición Holdout completada:")
    println("  • TRAIN: $(size(X_train, 1)) instancias ($(length(unique(subject_train))) sujetos)")
    println("  • TEST: $(size(X_test, 1)) instancias ($(length(unique(subject_test))) sujetos)")

    # Verificar distribución de clases
    println("\n  Distribución de clases en TRAIN:")
    for (clase, count) in sort(collect(countmap(y_train)))
        pct = count / length(y_train) * 100
        println("    • $clase: $count ($(round(pct, digits=1))%)")
    end
    
    println("\n  Distribución de clases en TEST:")
    for (clase, count) in sort(collect(countmap(y_test)))
        pct = count / length(y_test) * 100
        println("    • $clase: $count ($(round(pct, digits=1))%)")
    end

    return X_train, y_train, subject_train, X_test, y_test, subject_test
end

println("\n" * "-"^80)
println("PARTICIÓN HOLDOUT (10% TEST)")
println("-"^80)
X_train, y_train, subject_train, X_test, y_test, subject_test = holdout_split_subject_wise(X_full, y_full, subject_id_full);




--------------------------------------------------------------------------------
PARTICIÓN HOLDOUT (10% TEST)
--------------------------------------------------------------------------------
  Individuos seleccionados para TEST (holdout 10.0%): [18, 22, 25]

  Partición Holdout completada:
  • TRAIN: 9205 instancias (27 sujetos)
  • TEST: 1094 instancias (3 sujetos)

  Distribución de clases en TRAIN:
    • LAYING: 1734 (18.8%)
    • SITTING: 1593 (17.3%)
    • STANDING: 1696 (18.4%)
    • WALKING: 1546 (16.8%)
    • WALKING_DOWNSTAIRS: 1257 (13.7%)
    • WALKING_UPSTAIRS: 1379 (15.0%)

  Distribución de clases en TEST:
    • LAYING: 210 (19.2%)
    • SITTING: 184 (16.8%)
    • STANDING: 210 (19.2%)
    • WALKING: 176 (16.1%)
    • WALKING_DOWNSTAIRS: 149 (13.6%)
    • WALKING_UPSTAIRS: 165 (15.1%)


### 1.5  Validación cruzada individual-wise 

Crea k particiones para validación cruzada asegurando que todos los datos de un sujeto están en el mismo fold.


In [13]:
function individualwise_cv_splits(subject_id::AbstractVector; k=5)
    
    Random.seed!(SEED)
    
    subjects = unique(subject_id)
    n = length(subjects)
    
    shuffled = shuffle(subjects)

    # Dividir sujetos en k folds equilibrados
    folds = [shuffled[floor(Int, (i-1)*n/k)+1 : floor(Int, i*n/k)] for i in 1:k]

    splits = []
    
    for i in 1:k
        val_subjects = folds[i]
        train_subjects = vcat(folds[1:i-1]..., folds[i+1:end]...)
        
        is_val = [s in val_subjects for s in subject_id]
        is_train = .!is_val
        
        push!(splits, (
            is_train=is_train, 
            is_val=is_val,
            train_subjects=train_subjects,
            val_subjects=val_subjects
        ))
    end
    
    # Tabla
    println("\nFOLD | Train (muestras) | Val (muestras) | Train sujetos | Val sujetos")
    println("----------------------------------------------------------------------")
    for (i, split) in enumerate(splits)
        n_train = sum(split.is_train)
        n_val = sum(split.is_val)
        @printf("  %d  |       %4d       |      %4d      |       %2d      |     %2d      \n",
                i, n_train, n_val, length(split.train_subjects), length(split.val_subjects))
    end

    return splits
end

println("\n" * "-"^80)
println("VALIDACIÓN CRUZADA INDIVIDUAL-WISE (5-FOLD)")
println("-"^80)

cv_splits = individualwise_cv_splits(subject_train; k=5);



--------------------------------------------------------------------------------
VALIDACIÓN CRUZADA INDIVIDUAL-WISE (5-FOLD)
--------------------------------------------------------------------------------

FOLD | Train (muestras) | Val (muestras) | Train sujetos | Val sujetos
----------------------------------------------------------------------
  1  |       7378       |      1827      |       22      |      5      
  2  |       7565       |      1640      |       22      |      5      
  3  |       7103       |      2102      |       21      |      6      
  4  |       7503       |      1702      |       22      |      5      
  5  |       7271       |      1934      |       21      |      6      


### 1.6 Normalización

In [14]:
println("\n" * "-"^80)
println("NORMALIZACIÓN")
println("-"^80)

mutable struct MinMaxNormalizer <: Unsupervised end

function MLJBase.fit(normalizer::MinMaxNormalizer, verbosity::Int, X)
    X_matrix = MLJBase.matrix(X)
    
    # Calcular min y max por columna
    feature_min = vec(minimum(X_matrix, dims=1))
    feature_max = vec(maximum(X_matrix, dims=1))
    
    fitresult = (feature_min=feature_min, feature_max=feature_max)
    cache = nothing
    report = (feature_ranges = feature_max .- feature_min,)
    
    return fitresult, cache, report
end

function MLJBase.transform(normalizer::MinMaxNormalizer, fitresult, X)
    X_matrix = MLJBase.matrix(X)
    
    feature_min = fitresult.feature_min
    feature_max = fitresult.feature_max
    
    # Normalización Min-Max
    X_normalized = similar(X_matrix, Float64)
    
    for j in 1:size(X_matrix, 2)
        min_j = feature_min[j]
        range_j = feature_max[j] - min_j
        
        if range_j ≈ 0.0
            X_normalized[:, j] .= 0.0
        else
            X_normalized[:, j] = (X_matrix[:, j] .- min_j) ./ range_j
        end
    end
    
    return MLJBase.table(X_normalized)
end

# Registrar el transformador en MLJ
MLJBase.input_scitype(::Type{<:MinMaxNormalizer}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.output_scitype(::Type{<:MinMaxNormalizer}) = MLJBase.Table(MLJBase.Continuous)

println("\nMinMaxNormalizer registrado en MLJ: ", MinMaxNormalizer())


--------------------------------------------------------------------------------
NORMALIZACIÓN
--------------------------------------------------------------------------------

MinMaxNormalizer registrado en MLJ: MinMaxNormalizer()


## PARTE 2: Modelos básicos y selección de atributos

### 2.1 Filtrado de características
- Filtrado ANOVA
- Filtrado de Pearson
- Filtrado de Spearman
- Filtrado de Kendall Tau
- Filtrado por Mutual Information
- Filtrado RFE (con Logistic Regression, eliminando el 50 % de las variables 
en cada iteración)


Filtrado ANOVA

In [32]:
mutable struct ANOVAFilter <: Supervised
    k::Int
end

ANOVAFilter(; k::Int=50) = ANOVAFilter(k)

function MLJBase.fit(model::ANOVAFilter, verbosity::Int, X, y)
    Xmat = Float64.(MLJBase.matrix(X))
    y_numeric = Int.(levelcode.(y))
    
    n_features = size(Xmat, 2)
    fstats = zeros(Float64, n_features)
    
    for j in 1:n_features
        feature = Xmat[:, j]
        classes = unique(y_numeric)
        groups = [feature[y_numeric .== c] for c in classes]
        groups = filter(g -> length(g) > 0, groups)
        
        if length(groups) < 2
            fstats[j] = 0.0
            continue
        end
        
        try
            test = OneWayANOVATest(groups...)
            MSt = test.SStᵢ / test.DFt
            MSe = test.SSeᵢ / test.DFe
            fstats[j] = MSe > 0 ? MSt / MSe : 0.0
        catch
            fstats[j] = 0.0
        end
    end
    
    k_actual = min(model.k, n_features)
    idxs = sortperm(fstats, rev=true)[1:k_actual]
    
    fitresult = (idxs=idxs,)
    report = (fstats=fstats, selected_indices=idxs)
    
    return fitresult, nothing, report
end

function MLJBase.transform(model::ANOVAFilter, fitresult, X)
    Xmat = MLJBase.matrix(X)
    X_selected = Xmat[:, fitresult.idxs]
    return MLJBase.table(X_selected)
end

MLJBase.input_scitype(::Type{<:ANOVAFilter}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.target_scitype(::Type{<:ANOVAFilter}) = AbstractVector{<:Finite}
MLJBase.output_scitype(::Type{<:ANOVAFilter}) = MLJBase.Table(MLJBase.Continuous)

println("\nANOVAFilter registrado en MLJ: ", ANOVAFilter())


ANOVAFilter registrado en MLJ: ANOVAFilter(k = 50)


Filtrado de Pearson

In [33]:
mutable struct PearsonCorrelationFilter <: Supervised
    k::Int
end

PearsonCorrelationFilter(; k::Int=50) = PearsonCorrelationFilter(k)

function MLJBase.fit(model::PearsonCorrelationFilter, verbosity::Int, X, y)
    Xmat = Float64.(MLJBase.matrix(X))
    y_numeric = Int.(levelcode.(y))
    
    n_features = size(Xmat, 2)
    correlations = zeros(Float64, n_features)
    
    for j in 1:n_features
        try
            correlations[j] = cor(Xmat[:, j], y_numeric)
        catch
            correlations[j] = 0.0
        end
    end
    
    abs_correlations = abs.(correlations)
    k_actual = min(model.k, n_features)
    idxs = sortperm(abs_correlations, rev=true)[1:k_actual]
    
    fitresult = (idxs=idxs,)
    report = (correlations=correlations, selected_indices=idxs)
    
    return fitresult, nothing, report
end

function MLJBase.transform(model::PearsonCorrelationFilter, fitresult, X)
    Xmat = MLJBase.matrix(X)
    X_selected = Xmat[:, fitresult.idxs]
    return MLJBase.table(X_selected)
end

MLJBase.input_scitype(::Type{<:PearsonCorrelationFilter}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.target_scitype(::Type{<:PearsonCorrelationFilter}) = AbstractVector{<:Finite}
MLJBase.output_scitype(::Type{<:PearsonCorrelationFilter}) = MLJBase.Table(MLJBase.Continuous)

println("\nPearsonCorrelationFilter registrado en MLJ: ", PearsonCorrelationFilter())


PearsonCorrelationFilter registrado en MLJ: PearsonCorrelationFilter(k = 50)


Filtrado de Spearman

In [17]:
mutable struct SpearmanCorrelationFilter <: Supervised
    k::Int
end

SpearmanCorrelationFilter(; k::Int=50) = SpearmanCorrelationFilter(k)

function MLJBase.fit(model::SpearmanCorrelationFilter, verbosity::Int, X, y)
    Xmat = Float64.(MLJBase.matrix(X))
    y_numeric = Int.(levelcode.(y))
    
    n_features = size(Xmat, 2)
    correlations = zeros(Float64, n_features)
    
    for j in 1:n_features
        try
            correlations[j] = corspearman(Xmat[:, j], y_numeric)
        catch
            correlations[j] = 0.0
        end
    end
    
    abs_correlations = abs.(correlations)
    k_actual = min(model.k, n_features)
    idxs = sortperm(abs_correlations, rev=true)[1:k_actual]
    
    fitresult = (idxs=idxs,)
    report = (correlations=correlations, selected_indices=idxs)
    
    return fitresult, nothing, report
end

function MLJBase.transform(model::SpearmanCorrelationFilter, fitresult, X)
    Xmat = MLJBase.matrix(X)
    X_selected = Xmat[:, fitresult.idxs]
    return MLJBase.table(X_selected)
end

MLJBase.input_scitype(::Type{<:SpearmanCorrelationFilter}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.target_scitype(::Type{<:SpearmanCorrelationFilter}) = AbstractVector{<:Finite}
MLJBase.output_scitype(::Type{<:SpearmanCorrelationFilter}) = MLJBase.Table(MLJBase.Continuous)

println("\nSpearmanCorrelationFilter registrado en MLJ: ", SpearmanCorrelationFilter())


SpearmanCorrelationFilter registrado en MLJ: SpearmanCorrelationFilter(k = 50)


Filtrado de Kendall Tau 

In [18]:
mutable struct KendallCorrelationFilter <: Supervised
    k::Int
end

KendallCorrelationFilter(; k::Int=50) = KendallCorrelationFilter(k)

function MLJBase.fit(model::KendallCorrelationFilter, verbosity::Int, X, y)
    Xmat = Float64.(MLJBase.matrix(X))
    y_numeric = Int.(levelcode.(y))
    
    n_features = size(Xmat, 2)
    correlations = zeros(Float64, n_features)
    
    for j in 1:n_features
        try
            correlations[j] = corkendall(Xmat[:, j], y_numeric)
        catch
            correlations[j] = 0.0
        end
    end
    
    abs_correlations = abs.(correlations)
    k_actual = min(model.k, n_features)
    idxs = sortperm(abs_correlations, rev=true)[1:k_actual]
    
    fitresult = (idxs=idxs,)
    report = (correlations=correlations, selected_indices=idxs)
    
    return fitresult, nothing, report
end

function MLJBase.transform(model::KendallCorrelationFilter, fitresult, X)
    Xmat = MLJBase.matrix(X)
    X_selected = Xmat[:, fitresult.idxs]
    return MLJBase.table(X_selected)
end

MLJBase.input_scitype(::Type{<:KendallCorrelationFilter}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.target_scitype(::Type{<:KendallCorrelationFilter}) = AbstractVector{<:Finite}
MLJBase.output_scitype(::Type{<:KendallCorrelationFilter}) = MLJBase.Table(MLJBase.Continuous)

println("\nKendallCorrelationFilter registrado en MLJ: ", KendallCorrelationFilter())


KendallCorrelationFilter registrado en MLJ: KendallCorrelationFilter(k = 50)


Filtrado por Mutual Information

In [19]:
mutable struct MutualInformationFilter <: Supervised
    k::Int
    n_bins::Int
end

MutualInformationFilter(; k::Int=50, n_bins::Int=10) = MutualInformationFilter(k, n_bins)

function discrete_entropy(x::Vector)
    counts = countmap(x)
    probs = collect(values(counts)) ./ length(x)
    return -sum(p * log2(p) for p in probs if p > 0)
end

function mutual_information_score(x::Vector{Float64}, y::Vector{Int}; n_bins::Int=10)
    # Discretizar x
    x_min, x_max = extrema(x)
    if x_max ≈ x_min
        return 0.0
    end
    
    bin_edges = range(x_min, x_max, length=n_bins+1)
    x_discrete = [searchsortedfirst(bin_edges, val) - 1 for val in x]
    x_discrete = clamp.(x_discrete, 1, n_bins)
    
    # Calcular entropías
    H_x = discrete_entropy(x_discrete)
    H_y = discrete_entropy(y)
    
    # Entropía conjunta
    joint_counts = countmap(collect(zip(x_discrete, y)))
    joint_probs = collect(values(joint_counts)) ./ length(x)
    H_xy = -sum(p * log2(p) for p in joint_probs if p > 0)
    
    # Información mutua
    return H_x + H_y - H_xy
end

function MLJBase.fit(model::MutualInformationFilter, verbosity::Int, X, y)
    Xmat = Float64.(MLJBase.matrix(X))
    y_numeric = Int.(levelcode.(y))
    
    n_features = size(Xmat, 2)
    mi_scores = zeros(Float64, n_features)
    
    for j in 1:n_features
        try
            mi_scores[j] = mutual_information_score(Xmat[:, j], y_numeric; n_bins=model.n_bins)
        catch
            mi_scores[j] = 0.0
        end
    end
    
    k_actual = min(model.k, n_features)
    idxs = sortperm(mi_scores, rev=true)[1:k_actual]
    
    fitresult = (idxs=idxs,)
    report = (mi_scores=mi_scores, selected_indices=idxs)
    
    return fitresult, nothing, report
end

function MLJBase.transform(model::MutualInformationFilter, fitresult, X)
    Xmat = MLJBase.matrix(X)
    X_selected = Xmat[:, fitresult.idxs]
    return MLJBase.table(X_selected)
end

MLJBase.input_scitype(::Type{<:MutualInformationFilter}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.target_scitype(::Type{<:MutualInformationFilter}) = AbstractVector{<:Finite}
MLJBase.output_scitype(::Type{<:MutualInformationFilter}) = MLJBase.Table(MLJBase.Continuous)

println("\nMutualInformationFilter registrado en MLJ: ", MutualInformationFilter())


MutualInformationFilter registrado en MLJ: MutualInformationFilter(k = 50, …)


Filtrado RFE

In [20]:
mutable struct RFEFilter <: Supervised
    k::Int
    step::Float64  # Proporción de features a eliminar en cada iteración
end

RFEFilter(; k::Int=50, step::Float64=0.5) = RFEFilter(k, step)

function MLJBase.fit(model::RFEFilter, verbosity::Int, X, y)
    Xmat = Float64.(MLJBase.matrix(X))
    n_features = size(Xmat, 2)
    
    # Índices activos (empezamos con todos)
    active_idxs = collect(1:n_features)
    
    # Cargar modelo de Logistic Regression
    LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
    
    while length(active_idxs) > model.k
        # Entrenar modelo con características activas
        X_active = Xmat[:, active_idxs]
        
        lr_model = LogisticClassifier()
        mach = machine(lr_model, MLJBase.table(X_active), y)
        fit!(mach, verbosity=0)
        
        # Obtener importancias (magnitud de coeficientes)
        fitted_params = fitted_params(mach)
        coefs = fitted_params.coefs
        
        # Si coefs es una matriz (multiclase), usar norma por característica
        if coefs isa Matrix
            importances = vec(sum(abs.(coefs), dims=1))
        else
            importances = abs.(coefs)
        end
        
        # Eliminar el step% de características menos importantes
        n_to_eliminate = max(1, Int(floor(length(active_idxs) * model.step)))
        n_to_keep = max(model.k, length(active_idxs) - n_to_eliminate)
        
        keep_idxs = sortperm(importances, rev=true)[1:n_to_keep]
        active_idxs = active_idxs[keep_idxs]
    end
    
    fitresult = (idxs=active_idxs,)
    report = (selected_indices=active_idxs, n_selected=length(active_idxs))
    
    return fitresult, nothing, report
end

function MLJBase.transform(model::RFEFilter, fitresult, X)
    Xmat = MLJBase.matrix(X)
    X_selected = Xmat[:, fitresult.idxs]
    return MLJBase.table(X_selected)
end

MLJBase.input_scitype(::Type{<:RFEFilter}) = MLJBase.Table(MLJBase.Continuous)
MLJBase.target_scitype(::Type{<:RFEFilter}) = AbstractVector{<:Finite}
MLJBase.output_scitype(::Type{<:RFEFilter}) = MLJBase.Table(MLJBase.Continuous)

println("\nRFEFilter registrado en MLJ: ", RFEFilter())


RFEFilter registrado en MLJ: RFEFilter(k = 50, …)


In [21]:
# Verificar tipos y dimensiones
println("-"^80)
println("VERIFICACIÓN DE DATOS CARGADOS")
println("-"^80)

println("\n X_train:")
println("  • Tipo: ", typeof(X_train))
println("  • Dimensiones: ", size(X_train))
println("  • Elemento tipo: ", eltype(X_train))

println("\n y_train:")
println("  • Tipo: ", typeof(y_train))
println("  • Longitud: ", length(y_train))
println("  • Clases únicas: ", levels(y_train))
println("  • Distribución: ")
for (clase, count) in sort(collect(countmap(y_train)))
    pct = count / length(y_train) * 100
    println("    • $clase: $count ($(round(pct, digits=1))%)")
end

println("\n X_test:")
println("  • Dimensiones: ", size(X_test))

println("\n📊 y_test:")
println("  • Longitud: ", length(y_test))

println("\n📊 CV Splits:")
println("  • Número de folds: ", length(cv_splits))
for (i, split) in enumerate(cv_splits)
    n_train = sum(split.is_train)
    n_val = sum(split.is_val)
    println("  • Fold $i: Train=$n_train Val=$n_val")
end

--------------------------------------------------------------------------------
VERIFICACIÓN DE DATOS CARGADOS
--------------------------------------------------------------------------------

 X_train:
  • Tipo: Matrix{Float64}
  • Dimensiones: (9205, 561)
  • Elemento tipo: Float64

 y_train:
  • Tipo: CategoricalVector{String31, UInt32, String31, CategoricalValue{String31, UInt32}, Union{}}
  • Longitud: 9205
  • Clases únicas: String31["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS"]
  • Distribución: 
    • LAYING: 1734 (18.8%)
    • SITTING: 1593 (17.3%)
    • STANDING: 1696 (18.4%)
    • WALKING: 1546 (16.8%)
    • WALKING_DOWNSTAIRS: 1257 (13.7%)
    • WALKING_UPSTAIRS: 1379 (15.0%)

 X_test:
  • Dimensiones: (1094, 561)

📊 y_test:
  • Longitud: 1094

📊 CV Splits:
  • Número de folds: 5
  • Fold 1: Train=7378 Val=1827
  • Fold 2: Train=7565 Val=1640
  • Fold 3: Train=7103 Val=2102
  • Fold 4: Train=7503 Val=1702
  • Fold 5: Train=7271 Val=1

In [22]:
# Cargar modelos adicionales
using MLJMultivariateStatsInterface  # Para PCA, LDA, ICA
using MLJLinearModels                # Para LogisticClassifier (usado en RFE)
using MLJFlux                        # Para NeuralNetworkClassifier (MLP)
using Flux                           # Backend de redes neuronales

println("Paquetes de modelos cargados")

Paquetes de modelos cargados


In [23]:
# Cargar modelos de MLJ
KNNClassifier = @load KNNClassifier pkg=NearestNeighborModels verbosity=0
SVC = @load SVC pkg=LIBSVM verbosity=0
PCA_model = @load PCA pkg=MultivariateStats verbosity=0
LDA_model = @load LDA pkg=MultivariateStats verbosity=0
ICA_model = @load ICA pkg=MultivariateStats verbosity=0

println("Paquetes de técnicas de proyección cargadas")

Paquetes de técnicas de proyección cargadas


In [24]:
using Flux
using MLJFlux

mutable struct MLPBuilder <: MLJFlux.Builder
    hidden_layers::Vector{Int}
end

function MLJFlux.build(b::MLPBuilder, rng, n_in, n_out)
    init = Flux.glorot_uniform(rng)
    layers = []
    input_dim = n_in
    
    for hidden_dim in b.hidden_layers
        push!(layers, Dense(input_dim, hidden_dim, relu, init=init))
        input_dim = hidden_dim
    end
    push!(layers, Dense(input_dim, n_out, init=init))
    
    return Chain(layers...)
end

In [41]:
mutable struct FilterPipeline <: MLJBase.ProbabilisticNetworkComposite
    preprocessor
    selector
    classifier
end

function MLJBase.prefit(pipe::FilterPipeline, verbosity, X, y)
    Xs = source(X)
    ys = source(y)

    m_pre   = machine(:preprocessor, Xs)
    Z1         = MLJ.transform(m_pre, Xs)          

    m_selector = machine(:selector, Z1, ys)           # supervisado
    Z2         = MLJ.transform(m_selector, Z1)        

    m_clf      = machine(:classifier, Z2, ys)
    yhat       = MLJ.predict(m_clf, Z2)              

    return (
        predict = yhat,
        report  = (selector = node(report, m_selector),)
    )
end


In [38]:
function evaluate_pipeline(pipeline_name, pipeline_obj, X_train, y_train, cv_splits, X_test, y_test)
    println("\n" * "="^60)
    println("EVALUANDO: $pipeline_name")
    println("="^60)

    # 1. Validación Cruzada (CV)
    cv_accs = Float64[]
    
    for (i, split) in enumerate(cv_splits)
        # Índices lógicos a enteros
        train_idx = findall(split.is_train)
        val_idx = findall(split.is_val)
        
        # Entrenar
        mach = machine(pipeline_obj, X_train, y_train)
        fit!(mach, rows=train_idx, verbosity=0)
        
        # Predecir
        y_pred = predict_mode(mach, rows=val_idx)
        acc = accuracy(y_pred, y_train[val_idx])
        push!(cv_accs, acc)
        # print(".") # Progreso
    end
    
    mean_cv = mean(cv_accs)
    std_cv = std(cv_accs)
    println("\n  CV Accuracy (Mean ± Std): $(round(mean_cv, digits=4)) ± $(round(std_cv, digits=4))")
    
    # 2. Evaluación en Test (Entrenamiento con todo Train)
    mach_full = machine(pipeline_obj, X_train, y_train)
    fit!(mach_full, verbosity=0)
    y_test_pred = predict_mode(mach_full, X_test)
    test_acc = accuracy(y_test_pred, y_test)
    println("  Test Accuracy:            $(round(test_acc, digits=4))")
    
    return (mean_cv, std_cv, test_acc)
end

evaluate_pipeline (generic function with 1 method)

In [26]:
# -------------------------------------------------------------------------
# 2. DEFINICIÓN DE MODELOS BASE
# -------------------------------------------------------------------------

# MLP
mlp_50 = NeuralNetworkClassifier(builder=MLPBuilder([50]), epochs=20, batch_size=32)
mlp_100 = NeuralNetworkClassifier(builder=MLPBuilder([100]), epochs=20, batch_size=32)
mlp_100_50 = NeuralNetworkClassifier(builder=MLPBuilder([100, 50]), epochs=20, batch_size=32)

# KNN
knn_1 = KNNClassifier(K=1)
knn_10 = KNNClassifier(K=10)
knn_20 = KNNClassifier(K=20)

# SVM
svm_01 = SVC(cost=0.1)
svm_05 = SVC(cost=0.5)
svm_10 = SVC(cost=1.0);

### Pipelines MLP

In [42]:
results_mlp = DataFrame(
    Experiment = String[],
    Filter = String[],
    Projection = String[],
    Model = String[],
    CV_Mean = Float64[],
    CV_Std = Float64[],
    Test_Acc = Float64[]
);

# 1. MLP [50] + ANOVA + PCA
pipe_1 = FilterPipeline(
  Pipeline(normalizer=MinMaxNormalizer(), projection=PCA_model(maxoutdim=20)),
  ANOVAFilter(k=50),
  SVC(cost=1.0)
)

# # 2. MLP [100] + Pearson + ICA
# pipe_2 = FilteredPipe(
#     filter = PearsonCorrelationFilter(k=50),
#     pre = Pipeline(normalizer = MinMaxNormalizer(), projection = ICA_model(outdim=20)),
#     model = mlp_100
# )

# # 3. MLP [100,50] + RFE + Sin reducción
# pipe_3 = FilteredPipe(
#     filter = RFEFilter(k=50, step=0.5),
#     pre = Pipeline(normalizer = MinMaxNormalizer()),
#     model = mlp_100_50
# )

# Lista de experimentos
mlp_experiments = [
    ("Exp 1", pipe_1, "ANOVA", "PCA", "MLP[50]"),
    #("Exp 2", pipe_2, "Pearson", "ICA", "MLP[100]"),
    #("Exp 3", pipe_3, "RFE", "None", "MLP[100,50]")
]

# Evaluación
for (name, pipe, fname, pname, mname) in mlp_experiments
    mean_cv, std_cv, test_acc = evaluate_pipeline(
        "$name: $fname + $pname + $mname",
        pipe, X_train, y_train, cv_splits, X_test, y_test
    )
    push!(results_mlp, (name, fname, pname, mname, mean_cv, std_cv, test_acc))
end

# Mostrar resultados MLP
println("\nResultados MLP")
display(results_mlp)



EVALUANDO: Exp 1: ANOVA + PCA + MLP[50]


MethodError: MethodError: no method matching iterate(::CategoricalValue{String31, UInt32})
The function `iterate` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  iterate(!Matched::HTTP.WebSockets.WebSocket)
   @ HTTP C:\Users\aniba\.julia\packages\HTTP\ShTJs\src\WebSockets.jl:719
  iterate(!Matched::HTTP.WebSockets.WebSocket, !Matched::Any)
   @ HTTP C:\Users\aniba\.julia\packages\HTTP\ShTJs\src\WebSockets.jl:719
  iterate(!Matched::Combinatorics.CoolLexCombinations)
   @ Combinatorics C:\Users\aniba\.julia\packages\Combinatorics\nV2DW\src\combinations.jl:87
  ...


In [26]:
# -------------------------------------------------------------------------
# 3. EJECUCIÓN DE EXPERIMENTOS (PIPELINES)
# -------------------------------------------------------------------------

results_df = DataFrame(
    Experiment = String[],
    Filter = String[],
    Projection = String[],
    Model = String[],
    CV_Mean = Float64[],
    CV_Std = Float64[],
    Test_Acc = Float64[]
)

# Definir la lista de combinaciones
# Tuplas: (Nombre, Filtro, Proyección, Modelo)
experiments = [
    # 1. MLP [50] + ANOVA + PCA
    ("Exp 1", ANOVAFilter(k=50), PCA_model(maxoutdim=20), mlp_50, "ANOVA", "PCA", "MLP[50]"),
    
    # 2. MLP [100] + Pearson + ICA
    ("Exp 2", PearsonCorrelationFilter(k=50), ICA_model(outdim=20), mlp_100, "Pearson", "ICA", "MLP[100]"),
    
    # 3. MLP [100, 50] + RFE + Sin reducción
    ("Exp 3", RFEFilter(k=50, step=0.5), nothing, mlp_100_50, "RFE", "None", "MLP[100,50]"),
    
    # 4. KNN (k=10) + Spearman + LDA
    ("Exp 4", SpearmanCorrelationFilter(k=50), LDA_model(), knn_10, "Spearman", "LDA", "KNN(10)"),
    
    # 5. SVM (C=0.5) + Mutual Info + Sin reducción
    ("Exp 5", MutualInformationFilter(k=50), nothing, svm_05, "MutualInfo", "None", "SVM(0.5)"),
    
    # 6. KNN (k=1) + Kendall + PCA
    ("Exp 6", KendallCorrelationFilter(k=50), PCA_model(maxoutdim=20), knn_1, "Kendall", "PCA", "KNN(1)"),
    
    # 7. SVM (C=1.0) + Sin reducción + ICA
    ("Exp 7", nothing, ICA_model(outdim=20), svm_10, "None", "ICA", "SVM(1.0)"),
    
    # 8. KNN (k=20) + ANOVA + Sin reducción (Para cubrir k=20)
    ("Exp 8", ANOVAFilter(k=50), nothing, knn_20, "ANOVA", "None", "KNN(20)"),
    
    # 9. SVM (C=0.1) + Pearson + LDA (Para cubrir C=0.1)
    ("Exp 9", PearsonCorrelationFilter(k=50), LDA_model(), svm_01, "Pearson", "LDA", "SVM(0.1)")
]

for (name, filt, proj, model, fname, pname, mname) in experiments
    # Construcción dinámica del pipeline
    # MLJ ignora 'nothing' en el pipeline o podemos construirlo condicionalmente
    
    components = []
    push!(components, (:normalizer, MinMaxNormalizer()))
    
    if !isnothing(filt)
        push!(components, (:filter, filt))
    end
    
    if !isnothing(proj)
        push!(components, (:projection, proj))
    end
    
    push!(components, (:classifier, model))
    
    # Crear pipeline
    pipe = Pipeline(; components...)
    
    # Evaluar
    mean_cv, std_cv, test_acc = evaluate_pipeline("$name: $fname + $pname + $mname", 
                                                  pipe, X_train, y_train, cv_splits, X_test, y_test)
    
    # Guardar resultados
    push!(results_df, (name, fname, pname, mname, mean_cv, std_cv, test_acc))
end

# -------------------------------------------------------------------------
# 4. RESUMEN DE RESULTADOS
# -------------------------------------------------------------------------
println("\n" * "-"^80)
println("RESUMEN FINAL DE RESULTADOS - PARTE 2")
println("-"^80)
display(results_df)

# Ordenar por precisión en Test para ver el mejor
sort!(results_df, :Test_Acc, rev=true)
println("\nMejor modelo según Test Accuracy:")
println(results_df[1, :])

ArgumentError: ArgumentError: More than one supervised model in a pipeline is not permitted

In [ ]:

# Definición del filtro y modelo (asumiendo que mlp_100 es una instancia)
kendall_filter = KendallCorrelationFilter(k=100)
model_name = "MLP[100]"

println("\n" * "-"^80)
println("EXPERIMENTO 5: FILTRADO KENDALL (k=100) + MLP[100]")
println("-"^80)

# 1. Construir el pipeline usando el modelo compuesto (SupervisedPipeline)
# Esto permite encadenar el filtro (supervisado) con el clasificador (supervisado).
pipe_kendall = FilteredPipe(
    normalizer = MinMaxNormalizer(),
    filter = kendall_filter,
    projection = nothing, # Sin proyección adicional
    classifier = mlp_100
)

# 2. Evaluar el pipeline: la función unificada maneja CV y Test
# La función evaluate_pipeline devuelve (mean_cv, std_cv, test_acc)
mean_cv, std_cv, test_acc = evaluate_pipeline("Kendall(100) + None + $model_name", 
                                              pipe_kendall, X_train, y_train, cv_splits, X_test, y_test)

# 3. Mostrar y Guardar los resultados
println("\n  CV Accuracy: $(round(mean_cv, digits=4)) ± $(round(std_cv, digits=4))")
println("  Test Accuracy: $(round(test_acc, digits=4))")

# Guardar los resultados en el DataFrame (ajustado al formato de la tabla 'results_df')
push!(results_df, (
    "Exp 5",
    "Kendall(k=100)",
    "None",
    model_name,
    mean_cv,
    std_cv,
    test_acc
))

println("\nResultado de Experimento 5 guardado en results_df.")

UndefVarError: UndefVarError: `SupervisedPipeline` not defined in `Main`
Suggestion: check for spelling errors or missing imports.